In [1]:
############################################################################################################
# Overview: This script parses the data marts prepared using SQL and exports them as parquet data files.
############################################################################################################

In [ ]:
############################################################################################################
# Import packages
############################################################################################################
import datetime
import numpy as np
import os
import pandas as pd
import sqlalchemy
from itertools import product
from typing import Optional

In [ ]:
########################################################################################################################
# USER_SPECIFIC SETTING
# PROJECT_DATABASE: Name of the project database on the project server
# PAT_FILENAME: Name of the patient-level data mart
# ENC_FILENAME_1: Name of the encounter-level data mart (main copy)
# ENC_FILENAME_2: Name of the encounter-level data mart (additional variables)
# OUT_DIR_PATH: Path of the directory of the output datasets
########################################################################################################################
PROJECT_DATABASE: str = 'ProjectD52021'
PAT_FILENAME: str = 'BipolarDepression_PatientLevelVars'
ENC_FILENAME_1: str = 'BipolarDepression_EncLevelVars_v2'
ENC_FILENAME_2: str = 'BipolarDepression_AddlVarsOct2025'
OUT_DIR_PATH: str = '../00_Data/00_Raw_Data/'

In [ ]:
############################################################################################################
# Define SQL connection as a class for querying the database on the PROJECTS server
############################################################################################################

class SQLConnect:
    def __init__(self,
                 database: str):
        assert isinstance(database, str), 'database must be a string.'
        self.database: str = database

        # Set up the connection engine
        self.server: str = 'PROJECTS'
        self.driver_name: str = 'mssql+pyodbc'
        self.query_key: str = 'odbc connect'
        self.query_value: str = 'DRIVER={ODBC Driver 17 for SQL Server};'
        self.query_value = f'SERVER=tcp:{self.server};DATABASE={database};Trusted_Connection=yes;MARS_Connection=Yes;'
        self.connect_url = sqlalchemy.engine.URL.create(self.driver_name, query={self.query_key: self.query_value})
        self.engine = sqlalchemy.create_engine(self.connect_url, poolclass=sqlalchemy.pool.StaticPool,
                                               connect_args={"autocommit": True})

        self.connection = None

    def connect(self):
        self.connection = self.engine.connect().execution_options(isolation_level="AUTOCOMMIT")

    def disconnect(self):
        self.connection.close() if self.connection is not None else None
        self.engine.dispose()

    def data_mart_import(self,
                         data_mat_name: str,
                         schema_name: str = 'dbo',
                         n_rows: Optional[int] = None):
        """
        Import the data mart generated in the project database as a pandas.DataFrame.
        :param data_mat_name: A string.
            Name of the data mart.
        :param schema_name: A string.
            Name of the schema containing the data mart.
        :param n_rows: A positive integer or None.
            Number of rows to be imported. All rows will be imported if set as None.
        :return: A pandas.DataFrame. The data mart exported from the database.
        """
        row_str: str = '' if n_rows is None else f'TOP({n_rows})'
        sql_query: str = f'SELECT {row_str} * FROM {schema_name}.{data_mat_name}'
        df: pd.DataFrame = pd.read_sql_query(sqlalchemy.text(sql_query), con=self.connection)
        return df

In [ ]:
############################################################################################################
# Set up the SQL connection to the pre-specified database
############################################################################################################
C = SQLConnect(database=PROJECT_DATABASE)
C.connect()

In [ ]:
############################################################################################################
# Load the patient-level dataset
############################################################################################################
df_patient: pd.DataFrame = C.data_mart_import(data_mat_name=PAT_FILENAME, n_rows=None)
print(f'Dimension of the full patient-level data mart: {df_patient.shape}')

In [ ]:
############################################################################################################
# Load the encounter-level datasets
############################################################################################################
df_encounter_1: pd.DataFrame = C.data_mart_import(data_mat_name=ENC_FILENAME_1, n_rows=None)
df_encounter_2: pd.DataFrame = C.data_mart_import(data_mat_name=ENC_FILENAME_2, n_rows=None)
df_encounter = pd.merge(left=df_encounter_1, right=df_encounter_2, on=['PatientDurableKey', 'EncounterKey', 'EncDate'], how='outer')
df_encounter.drop(columns=['HxSuicideAttempt60DaysPrior'], inplace=True, errors='ignore')
print(f'Dimension of the full encounter-level data mart: {df_encounter.shape}')

In [ ]:
############################################################################################################
# Export the patient-level and encounter-level datasets
############################################################################################################
os.makedirs(OUT_DIR_PATH, exist_ok=True)

pat_filepath: str = os.path.join(OUT_DIR_PATH, 'Patient_full.parquet')
df_patient.to_parquet(pat_filepath)
print(f'Patient-level data saved.')

enc_filepath: str = os.path.join(OUT_DIR_PATH, 'Encounter_full.parquet')
df_encounter.to_parquet(enc_filepath)
print(f'Encounter-level data saved.')

In [ ]:
############################################################################################################
# Close the SQL connection to preserve resources on the DSVM & SQL server
############################################################################################################
C.disconnect()